In [ ]:
!pip install -q faiss-cpu faiss-gpu sentence-transformers datasets

In [2]:
import shutil
import urllib.request as request
from contextlib import closing

# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)
import tarfile

# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()
import numpy as np

# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')
# data we will search through
xb = read_fvecs('./sift/sift_base.fvecs')  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('./sift/sift_query.fvecs')
# take just one query (there are many in sift_learn.fvecs)
xq = xq[0].reshape(1, xq.shape[1])

In [3]:
xb.shape, xq.shape

((1000000, 128), (1, 128))

In [4]:
import faiss

d = xb.shape[1]
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, 256)

In [5]:
index_f = faiss.index_factory(d, 'IVF256,Flat')

In [6]:
k = 100
dist, idx = index.search(xq, k=k)

In [7]:
dist_f, idx_f = index_f.search(xq, k=k)

In [8]:
idx_f.tolist() == idx.tolist()

True

In [9]:
d = xb.shape[1]
m = 32
nbits = 8
nlist = 256

opq = faiss.OPQMatrix(d, m)
vecs = faiss.IndexFlatL2(d)
sub_index = faiss.IndexIVFPQ(vecs, d, nlist, m , nbits)
index = faiss.IndexPreTransform(opq, sub_index)
index = faiss.IndexRefineFlat(index)
index.train(xb)
index.add(xb)

In [10]:
d = xb.shape[1]
index = faiss.index_factory(d, "OPQ32,IVF256,PQ32,RFlat")
index.train(xb)
index.add(xb)